In [1]:
from qdrant_client import QdrantClient, models
c = QdrantClient("http://localhost:6333")

# 看看集合有幾筆
print(c.count("docs_hybrid", exact=True).count)

# 抽樣看 payload（只看 10 筆）
r = c.scroll(
    collection_name="docs_hybrid",
    limit=10,
    with_payload=True,
    with_vectors=False,
)
for pt in r[0]:
    print(pt.id, pt.payload.get("source"))

c:\work\chiayi_customer_service\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


5
3c4f3703-7500-47e3-b608-87fb85df6a59 docs\qa.md
4dd86cab-ba61-41a0-a913-3dadb3be87ab docs\qa.md
63dbcf20-6ba2-44a6-a58f-dce9e0a952f0 docs\qa.md
852dc10f-a6bb-4835-91a3-02f6f247e69d docs\qa.md
9d59b122-a7fc-47ac-972e-83bcc193edf6 docs\qa.md


In [2]:
r = c.query_points(
    collection_name="docs_hybrid",
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    prefetch=[
      models.Prefetch(query=models.Document(text="*", model="Qdrant/bm25"), using="sparse"),
      models.Prefetch(query=models.Document(text="*", model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"), using="dense"),
    ],
    limit=5,
    with_payload=True,
    query_filter=models.Filter(
        must=[models.FieldCondition(key="source", match=models.MatchText(text="qa.md"))]
    ),
)
print(len(r.points))


c:\work\chiayi_customer_service\venv\Lib\site-packages\qdrant_client\embed\embedder.py:78: UserWarning: The model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  model = TextEmbedding(model_name=model_name, **options)


5
